In [2]:
import pandas as pd
import numpy as np
import altair as alt
import janitor
from datetime import datetime, timedelta
import calendar
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
alt.renderers.enable('default')
import re
import nltk
from collections import Counter

import altair as alt
#nltk.download()
alt.data_transformers.enable('json')

DataTransformerRegistry.enable('json')

In [3]:
cleaned_english_resume = pd.read_csv('C:/Users/NIW/Documents/Capstone Project/data/clean_data/05182020_cleaned_english_resumes_V1.0.csv',index_col = 0)

In [4]:
cleaned_english_resume.sample(n=5)

,employee_name,employee_code,store,raw_resume,resume_text,resume_bline,language,file_type,clean_text
339,"Phillips, Dominick",MZN,WW 248,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Dominick Phillips Phone: (780)-228-6706 E-Mai...,"['', '', '', '', '', '', '', '', '', '', '', '...",English,docx,email current objective customer serviceorient...
475,"Wylie, Kevin",N6W,WE 1273,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"Kevin a. wylie 311 Park St. West, Dundas. Ont...","['', '', '', '', '', '', '', '', '', '', '', '...",English,docx,wylie Park creative individual natural leader ...
184,"Hussain, Syed Salman",MVK,WE 535,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,MAY 2014 MAY 2013 - JULY 2015 Ajeeb Apparel i...,"['', '', '', '', '', '', '', '', '', '', '', '...",English,pdf,JULY Ajeeb Apparel start clothing brand found ...
6,"Ahammed, Kawser",NBQ,WW 284,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"Kawser Ahammed 414-1225 Moss Avenue, Saskatoo...","['', '', '', '', '', '', '', '', '', '', '', '...",English,docx,Moss Avenue HIGHLIGHTS QUALIFICATIONS year dir...
122,"Dor-Traynor, Corey",MWK,WW 330,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"COREY DOR-TRAYNOR 57 COLLIN COURT, RICHMOND H...","['', '', '', '', '', '', '', '', '', '', '', '...",English,docx,COLLIN court RICHMOND HILL look opportunity de...


In [14]:
sample_test = cleaned_english_resume[cleaned_english_resume.employee_code.isin(['MQN','N2Q','MQK','N9B','MYB'])][['employee_name','employee_code','file_type','resume_text','raw_resume','resume_bline','clean_text']]
sample_test_2 = cleaned_english_resume[cleaned_english_resume.employee_code.isin(['N5A','MWK','N4U','N69','N1L'])][['employee_name','employee_code','file_type','resume_text','raw_resume','resume_bline','clean_text']]
sample_test_2 = cleaned_english_resume.sample(n=10,random_state=4321)[['employee_name','employee_code','file_type','resume_text','raw_resume','resume_bline','clean_text']]

In [294]:
def segment_jump(line):
    if "volunteer"  in line and len(line.split()) < 3:
        section = "volunteering"
    elif ('exp' in line and len(line.split()) < 3) or ('history' in line) or ('work experience' in line):
        section = "work experience"
    elif 'education' in line: #and len(line.split()) < 3:
        section = "education"
    elif "skill" in line and len(line.split()) < 5:
        section = "skills"
    elif 'references' in line and len(line.split()) < 3:
        section = "references"
    elif 'curricular' in line and len(line.split()) < 4:
        section = "extra-curricular activities"
    elif 'summary' in line and len(line.split()) < 4:
        section = "Summary"
    elif 'interests' in line and len(line.split()) < 3:
        section = "interests"
    else:
        section = "unidentified"
    return section

In [295]:
# Creating an empty dict 
def resume_segmentation(resume_text):
    resume_text = resume_text.lower()  
    #resume_text = resume_text.split("\n")
    resume_text = re.split('\n|  |   ',resume_text)
    myDict = dict() 
    current_section = "start"
    # Iterating for each line in the resume tex
    for each_line in resume_text:
        if len(each_line) < 2:
            continue
        #each_line = each_line.replace("'","")
        #print("start")
        #print(each_line)
        #print("end")
        #print(each_line)
        #print(segment_jump(each_line))
        each_line = each_line.replace("\t","")
        each_line = each_line.replace("\t","")
        if segment_jump(each_line)!="unidentified":
            current_section = segment_jump(each_line)
        else:
            myDict.setdefault(current_section, []).append(each_line)
    return myDict

In [296]:
sample_test_2.head(5)

,employee_name,employee_code,file_type,resume_text,raw_resume,resume_bline,clean_text,segmented_resume
231,"Kumar, Ashok",N5K,pdf,"ASHOK KUMAR 2910 Highway 7 1803, Expo City to...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",Highway Expo City tower Mobile objective secur...,"{'start': [' ashok kumar', '2910 highway 7', '..."
99,"Choi, Nathan",N1U,docx,NATHAN CHOI Phone: (778) 987-1311 Email: nath...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",PROFILE bear raise enjoy sale challenge reward...,"{'start': ['nathan choi', 'phone: (778) 987-13..."
294,"Molnar, Connor",KVU,doc,Professional Resume Connor Molnar 384 Rogers ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",Professional Resume Rogers st responsible hard...,"{'start': ['professional resume', 'connor moln..."
205,"Kaur, Ramandeep",MYH,docx,"Ramandeep Kaur 2830 E Arens Road, Suite #306 ...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",Arens Road Suite objective look invest organiz...,"{'start': ['ramandeep kaur', '2830 e arens roa..."
391,"Shah, Shreyansh",N70,docx,"shreyansh Shah 1510-125 PARKWAY FOREST DRIVE,...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",shreyansh PARKWAY FOREST DRIVE NORTH YORK look...,"{'start': ['shreyansh shah', '1510-125 parkway..."


In [297]:
sample_test_2['segmented_resume'] = sample_test_2.raw_resume.apply(lambda x: resume_segmentation(x))

In [298]:
sample_test_2

,employee_name,employee_code,file_type,resume_text,raw_resume,resume_bline,clean_text,segmented_resume
231,"Kumar, Ashok",N5K,pdf,"ASHOK KUMAR 2910 Highway 7 1803, Expo City to...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",Highway Expo City tower Mobile objective secur...,"{'start': [' ashok kumar', '2910 highway 7', '..."
99,"Choi, Nathan",N1U,docx,NATHAN CHOI Phone: (778) 987-1311 Email: nath...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",PROFILE bear raise enjoy sale challenge reward...,"{'start': ['nathan choi', 'phone: (778) 987-13..."
294,"Molnar, Connor",KVU,doc,Professional Resume Connor Molnar 384 Rogers ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",Professional Resume Rogers st responsible hard...,"{'start': ['professional resume', 'connor moln..."
205,"Kaur, Ramandeep",MYH,docx,"Ramandeep Kaur 2830 E Arens Road, Suite #306 ...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",Arens Road Suite objective look invest organiz...,"{'start': ['ramandeep kaur', '2830 e arens roa..."
391,"Shah, Shreyansh",N70,docx,"shreyansh Shah 1510-125 PARKWAY FOREST DRIVE,...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",shreyansh PARKWAY FOREST DRIVE NORTH YORK look...,"{'start': ['shreyansh shah', '1510-125 parkway..."
389,"Shah, Sheena",NCN,docx,"SHEENA SHAH APT#B103, 7110-41 STREET EMAIL: s...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",STREET email LLOYDMINSTER career objective sal...,"{'start': ['sheena shah', 'apt#b103, 7110-41 s..."
42,"Ayoub, Omar",MRC,pdf,Microsoft Word - Omar Ayoub Resume (11-2018) ...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",Microsoft Word Resume Born March Nationality S...,{'start': ['microsoft word - omar ayoub resume...
89,"Carr, Daniel",MY9,pdf,Indeed Resume Daniel Carr Experienced Marketi...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",Resume experienced marketing sale People Manag...,"{'start': ['indeed resume', 'daniel carr', 'ex..."
195,"Joshi, Sahil",MQ2,docx,"SAHIL JOSHI joshi_sahil@hotmail.com | Surrey,...",\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",SUMMARY Experienced Sales Manager retail indus...,"{'start': ['sahil joshi', 'joshi_sahil@hotmail..."
428,"Streilein, Dustin",MWM,docx,Dustin J. Streilein 217 – 1465 K.L.O Road 250...,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"['', '', '', '', '', '', '', '', '', '', '', '...",Road objective obtain fulltime position grow o...,"{'start': ['dustin j. streilein', '217 – 1465 ..."


In [299]:
type(sample_test_2[4:5]['segmented_resume'].iloc[0])

dict

In [300]:
sample_test_2[4:5]['raw_resume'].iloc[0]

"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\tshreyansh Shah\n1510-125 PARKWAY FOREST DRIVE, NORTH YORK, ON, M2J 1L9\n\n\tPhone: +1(306) -941-1983 • E-mail: shreyanshshah3609@gmail.com\n\n\nObjective\nLooking forward to a Management position with an esteemed organization whereby my skills and abilities can be utilized and enhanced to the mutual benefit of the individual and organization.\n\nPROFILE SUMMARY\n· Team player with positive attitude\n· Willing and adaptable to quickly learn new technologies\n· Dedicated toward team collaboration to pursue increase in productivity and results\xa0\n· Self-motivated achiever to succeed beyond expectations\n· Excellent problem solving, analytical reasoning, and time management skills with superior attention to detail\n\nCOMPUTER SKILLS\xa0\n· POS+, Client Tracking System, Microsoft Office including Word, Excel, PowerPoint, Access, Outlook\n\nPROFESSIONAL EXPERIENCE\n\nFairstone Financial, Nort

In [301]:
sample_dict = sample_test_2[4:5]['segmented_resume'].iloc[0]
sample_dict

{'start': ['shreyansh shah',
  '1510-125 parkway forest drive, north york, on, m2j 1l9',
  'phone: +1(306) -941-1983 • e-mail: shreyanshshah3609@gmail.com',
  'objective',
  'looking forward to a management position with an esteemed organization whereby my skills and abilities can be utilized and enhanced to the mutual benefit of the individual and organization.'],
 'Summary': ['· team player with positive attitude',
  '· willing and adaptable to quickly learn new technologies',
  '· dedicated toward team collaboration to pursue increase in productivity and results\xa0',
  '· self-motivated achiever to succeed beyond expectations',
  '· excellent problem solving, analytical reasoning, and time management skills with superior attention to detail'],
 'skills': ['· pos+, client tracking system, microsoft office including word, excel, powerpoint, access, outlook'],
 'work experience': ['fairstone financial, north york, on',
  ' may 2019 - present',
  ' lending specialist 1',
  '· start the

In [302]:
work_exp = sample_dict['work experience']

In [303]:
for each_line  in work_exp:
    tokenize_line = nltk.word_tokenize(each_line)
    line_tags = nltk.pos_tag(tokenize_line)
    print(tokenize_line)
    print(line_tags)

['fairstone', 'financial', ',', 'north', 'york', ',', 'on']
[('fairstone', 'NN'), ('financial', 'JJ'), (',', ','), ('north', 'JJ'), ('york', 'NN'), (',', ','), ('on', 'IN')]
['may', '2019', '-', 'present']
[('may', 'MD'), ('2019', 'CD'), ('-', ':'), ('present', 'NN')]
['lending', 'specialist', '1']
[('lending', 'VBG'), ('specialist', 'NN'), ('1', 'CD')]
['·', 'start', 'the', 'day', 'with', 'managers', 'meeting', 'and', 'report', 'the', 'previous', 'day', 'activity', '.']
[('·', 'JJ'), ('start', 'VBP'), ('the', 'DT'), ('day', 'NN'), ('with', 'IN'), ('managers', 'NNS'), ('meeting', 'VBG'), ('and', 'CC'), ('report', 'VB'), ('the', 'DT'), ('previous', 'JJ'), ('day', 'NN'), ('activity', 'NN'), ('.', '.')]
['·', 'call', 'members', 'to', 'update', 'the', 'web', 'application', 'filled', 'by', 'them', ',', 'pull', 'credit', 'bureau', ',', 'and', 'book', 'loan', '.']
[('·', 'NN'), ('call', 'NN'), ('members', 'NNS'), ('to', 'TO'), ('update', 'VB'), ('the', 'DT'), ('web', 'JJ'), ('application', 'N

In [304]:
test = sample_test_2[4:5]['raw_resume'].iloc[0]
test

"\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\tshreyansh Shah\n1510-125 PARKWAY FOREST DRIVE, NORTH YORK, ON, M2J 1L9\n\n\tPhone: +1(306) -941-1983 • E-mail: shreyanshshah3609@gmail.com\n\n\nObjective\nLooking forward to a Management position with an esteemed organization whereby my skills and abilities can be utilized and enhanced to the mutual benefit of the individual and organization.\n\nPROFILE SUMMARY\n· Team player with positive attitude\n· Willing and adaptable to quickly learn new technologies\n· Dedicated toward team collaboration to pursue increase in productivity and results\xa0\n· Self-motivated achiever to succeed beyond expectations\n· Excellent problem solving, analytical reasoning, and time management skills with superior attention to detail\n\nCOMPUTER SKILLS\xa0\n· POS+, Client Tracking System, Microsoft Office including Word, Excel, PowerPoint, Access, Outlook\n\nPROFESSIONAL EXPERIENCE\n\nFairstone Financial, Nort

In [305]:
test = "hy my name is,Manuel"

In [306]:
###### Work Experience Line Labeling

In [307]:
all_resume = cleaned_english_resume

In [308]:
training_df = pd.read_csv('C:/Users/NIW/Documents/Capstone Project/data/clean_data/manual_clean_training_dataset_v7.csv')

In [309]:
resume_training = pd.merge(all_resume,training_df[['employee_code']],how = "inner")

In [310]:
resume_training['segmented_resume'] = resume_training.raw_resume.apply(lambda x: resume_segmentation(x))

In [311]:
resume_training.head(5)

,employee_name,employee_code,store,raw_resume,resume_text,resume_bline,language,file_type,clean_text,segmented_resume
0,"Abdi, Melika",N3O,TB 146,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Skills Experience MELIKA ABDI Carmelo Avenue ...,"['', '', '', '', '', '', '', '', '', '', '', '...",English,pdf,Skills Experience Carmelo Avenue Ph ability wo...,"{'work experience': ['melika abdi', 'carmelo a..."
1,"Abdulali, Kaizar",BDP,WW 252,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,KAIZAR ABDULALI kaizar83@gmail.com | 780 990-...,"['', '', '', '', '', '', '', '', '', '', '', '...",English,pdf,thank consideration role Project Manager Busin...,"{'start': ['kaizar abdulali ', 'kaizar83@gmail..."
2,"Abrams, James",MZV,WW 851,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"Michael Abrams 5214 Gerrish Street, Apt 221 C...","['', '', '', '', '', '', '', '', '', '', '', '...",English,docx,Gerrish Apt entrepreneur new customer acquisit...,"{'start': ['michael abrams ', '5214 gerrish st..."
3,"Abro, Muhammad Wali",N62,WW 394,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"WALI ABRO 4 Montpelier Street, Brampton, ON |...","['', '', '', '', '', '', '', '', '', '', '', '...",English,pdf,Montpelier OVERVIEW Languages Fluent English S...,"{'start': ['wali abro ', '4 montpelier street,..."
4,"Abrum, Nabil",N7R,WE 540,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Communicating Your Value Proposition: Nabil A...,"['', '', '', '', '', '', '', '', '', '', '', '...",English,doc,communicate value proposition Beryl Private ob...,{'start': ['communicating your value propositi...


In [312]:
MZW

NameError: name 'MZW' is not defined

In [313]:
resume_training['work_experience'] = resume_training.segmented_resume.apply(lambda x : x.get('work experience'))
resume_training['education'] = resume_training.segmented_resume.apply(lambda x : x.get('education'))

In [355]:
resume_training[['employee_code','work_experience','education']].sample(15)

,employee_code,work_experience,education
182,MVX,None,None
161,N4U,"[angles hair design, winnipeg manitoba , march...","[sanford collegiate, sanford manitoba , gradua..."
58,MY5,"[assistant manager, wow! mobile boutique - cam...","[audio engineering graduate, ontario institute..."
236,MX9,"[ front desk clerk/ night auditor ​, 05/2013 t...","[ontario college diploma​: 2013 , georgian col..."
228,N3E,[most recent position: two years as store mana...,[· niagara falls college graduate with special...
249,N3M,"[ceo, fabutan & hush lash studio - lloydminste...","[computer training, 3a academy employment and ..."
76,N67,"[sales associate/manager on duty, fido, respon...",None
284,MQF,"[counter manager of ysl, hudson bay rideau bay...","[ningbo dahongying university (2008-2012) , b..."
16,MRN,"[continued, c h a i r m a n s, m a n u f a c t...","[business acquired at capilano university, com..."
83,ND9,"[save on foods , 1800 tranquille rd, kamloops ...",None


In [384]:
resume_training['flag'] = resume_training.apply(lambda x: flag_undf(x),axis=1)

In [385]:
resume_training.head(5)

,employee_name,employee_code,store,raw_resume,resume_text,resume_bline,language,file_type,clean_text,segmented_resume,work_experience,education,flag
0,"Abdi, Melika",N3O,TB 146,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Skills Experience MELIKA ABDI Carmelo Avenue ...,"['', '', '', '', '', '', '', '', '', '', '', '...",English,pdf,Skills Experience Carmelo Avenue Ph ability wo...,"{'work experience': ['melika abdi', 'carmelo a...","[melika abdi, carmelo avenue, coquitlam, bc v3...","[marketing information., inform customers of e...",segmented
1,"Abdulali, Kaizar",BDP,WW 252,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,KAIZAR ABDULALI kaizar83@gmail.com | 780 990-...,"['', '', '', '', '', '', '', '', '', '', '', '...",English,pdf,thank consideration role Project Manager Busin...,"{'start': ['kaizar abdulali ', 'kaizar83@gmail...",[sales manager (samsung experience store) nov ...,"[bachelor of arts – business administration , ...",segmented
2,"Abrams, James",MZV,WW 851,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"Michael Abrams 5214 Gerrish Street, Apt 221 C...","['', '', '', '', '', '', '', '', '', '', '', '...",English,docx,Gerrish Apt entrepreneur new customer acquisit...,"{'start': ['michael abrams ', '5214 gerrish st...","[novama consulting inc., halifax, nova scotia,...","[high school diploma , notre dame catholic sec...",segmented
3,"Abro, Muhammad Wali",N62,WW 394,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,"WALI ABRO 4 Montpelier Street, Brampton, ON |...","['', '', '', '', '', '', '', '', '', '', '', '...",English,pdf,Montpelier OVERVIEW Languages Fluent English S...,"{'start': ['wali abro ', '4 montpelier street,...",None,[athabasca university class of 2021 (part-time...,not segmented
4,"Abrum, Nabil",N7R,WE 540,\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n...,Communicating Your Value Proposition: Nabil A...,"['', '', '', '', '', '', '', '', '', '', '', '...",English,doc,communicate value proposition Beryl Private ob...,{'start': ['communicating your value propositi...,"[best buy canada, ottawa, ontario, may 2012 - ...","[carleton university, ottawa, ontario, present...",segmented


In [386]:
not_found = resume_training.groupby(['flag']).agg({"employee_code":'count'})

In [394]:
not_found['perc'] = (not_found['employee_code']/not_found['employee_code'].sum(axis = 0)).round(2)

In [396]:
not_found.rename(columns = {'employee_code':"count"})

,count,perc
flag,,
not segmented,58,0.2
segmented,230,0.8


In [383]:
def flag_undf(df):
    if df['work_experience'] == None or df['education'] == None:
        return "not segmented"
    else:
        return "segmented"

In [352]:
resume_training[resume_training['employee_code']=='N69']['raw_resume'].iloc[0]

'\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nMuhammad Siraj Ud Din\n1073 Nadalin Heights – Milton, ON L9T 8H4\nPhone: (306)-737-3907         Email: sirajuddin591@gmail.com\n\n\nSKILLS & Competencies\n· Experienced in public relations, rapport building, coordination, collaboration and team-work\n· Strong analytical and mathematical skills\n· Ability to identify issues, propose and implement solutions \n· Excellent verbal and communication skills attained by extensive client dealings \n· Proficient in MS Word, Excel and PowerPoint\n· Strong interpersonal, and communication skills\n· Highly creative and organized\n· Positive energy, “can-do” attitude, adaptive and high degree of initiative\n\n\nWORK EXPERIENCE\n\nTD CANADA TRUST                                                                                                          Oct’18-Present   \nCustomer Experience Associate\n· Efficiently handle customers by getting the task do

In [350]:
for i in resume_training[resume_training['employee_code']=='N69']['raw_resume'].iloc[0].split('\n'):
    if len(i)<2:
        continue
    else:
        print(i)

Microsoft Word - Omar Ayoub Resume (11-2018) copy.docx
   
1 M           
  
Name            Omar Ayoub 
Born            23rd March 1993 
Nationality           Swiss, Canadian PR 
Languages           English, French, German, Arabic 
Education           MIA, April 2021 
Phone                             +1 (613) 791-9250 
Email                             omar.b.ayoub@gmail.com 
Information  
Cubase 
Wwise                           
Garage Band  
Adobe Photoshop Lightroom 
Python 
C++ 
JavaScript 
SOFTWARE SKILLS 
BestBuy CA   
District employee of the month, Aug 
2016 
Employee of the month, May, June 2016  
AWARDS 
RESUME 
Sept 2018-  Music Industry Arts. 
Present Dep. Of Art Media and Design 
Algonquin College 
Sept 2013-            Bachelor of Computer Science 
April 2016 Software Engineering stream 
 Carleton University 
  
Oct 2017- Vidéotron LTEE 
Present Business Solution Expert 
 Catering the wireless services needs of small to 
medium business owners. 
After sale support for a

In [347]:
resume_training[resume_training['employee_code']=='MRC']['segmented_resume'].iloc[0]

{'start': ['microsoft word - omar ayoub resume (11-2018) copy.docx',
  '1 m',
  'name',
  'omar ayoub ',
  'born',
  '23rd march 1993 ',
  'nationality',
  ' swiss, canadian pr ',
  'languages',
  ' english, french, german, arabic '],
 'education': [' mia, april 2021 ',
  'phone',
  ' +1 (613) 791-9250 ',
  'email',
  ' omar.b.ayoub@gmail.com ',
  'information',
  'cubase ',
  'wwise',
  'garage band',
  'adobe photoshop lightroom ',
  'python ',
  'c++ ',
  'javascript ',
  'resilient ',
  'team player ',
  'critical thinker',
  'creative ',
  'future focused '],
 'skills': ['bestbuy ca',
  'district employee of the month, aug ',
  '2016 ',
  'employee of the month, may, june 2016',
  'awards ',
  'resume ',
  'sept 2018-',
  'music industry arts. ',
  'present dep. of art media and design ',
  'algonquin college ',
  'sept 2013-',
  'bachelor of computer science ',
  'april 2016 software engineering stream ',
  ' carleton university ',
  'oct 2017- vidéotron ltee ',
  'present busine

In [344]:
unit_sample = resume_training.sample(1)
print(unit_sample['employee_code'])
unit_sample['education'].iloc[0]

147    LSE
Name: employee_code, dtype: object


['while providing ample opportunities towards continuous learning and advancement. ',
 '• passed class x- hartleys high school, kolkata : 65% ',
 '• passed class xii higher secondary examination (10+2)- commerce : (first division) 73% ',
 '• graduation – ba llb, university of calcutta, india (2009-2014) : first class ',
 '• masters - llm in international commercial law – university of sussex (2014-2015, ',
 'sussex india scholarship): first class ',
 '• institute of company secretaries of india (icsi)- foundation cleared ',
 '• business administration – human resource and leadership (present) ',
 'scholarship: ',
 'sussex india scholarship of 3000 gbp for the year 2014-15. ']

In [324]:
resume_training[['employee_code','work_experience']].sample()

,employee_code,work_experience
0,N3O,"[melika abdi, carmelo avenue, coquitlam, bc v3..."


In [346]:
resume_training[resume_training['employee_code']=='MRC']['work_experience'].iloc[0]

In [118]:
type(resume_training[['work_experience']].iloc[0][0])

list

In [119]:
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import string
def clean(doc):
    stop = set(stopwords.words('english'))
    exclude = set(string.punctuation+"·"+"−"+"§"+"•"+"–"+"––"+"✓"+"•"+"â—"+"●")
    lemma = WordNetLemmatizer()
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word) for word in punc_free.split())
    #doc_clean = normalized.split() 
    return normalized

In [48]:
### Corpus generation for topic modeling

In [49]:
corpus_lines_we = list()
corpus_lines_we_clean = list()
for i in resume_training['work_experience']:
    if i == None:
        continue
    for j in i:
        #if len(j) <4:
        #    continue
        if len(j.split())>7:
            continue
        corpus_lines_we.append(j)
        corpus_lines_we_clean.append(clean(j))
        #print("------")
        #print(len(j))
        #print(j)
        #print("---end----")

In [50]:
pd.DataFrame({"original":corpus_lines_we,"clean":corpus_lines_we_clean}).sample(n = 150,random_state = 1234).to_csv('../../../data/clean_data/automation_work_exp_training.csv')

FileNotFoundError: [Errno 2] No such file or directory: '../../../data/clean_data/automation_work_exp_training.csv'

In [90]:
print(pd.DataFrame({"original":corpus_lines_we,"clean":corpus_lines_we_clean}).sample(n = 150,random_state = 1234).iloc[0])

original    - frite arseneault
clean         frite arseneault
Name: 268, dtype: object


In [91]:
#### Training Labelling Model

In [94]:
data = pd.read_csv('../../../data/clean_data/automation_work_exp_training_labeled.csv',index_col = 0)

In [95]:
data

,original,clean,label
268,- frite arseneault,frite arseneault,company
3011,focus on retail sales of wireless,focus retail sale wireless,resp
3551,• performing database entry.,performing database entry,resp
1619,april 2018 - present,april 2018 present,date
3712,2012-2013,20122013,date
...,...,...,...
1749,"peterborough, on",peterborough,other
436,nestle india,nestle india,company
417,customer service representative,customer service representative,position
3633,1. consumer sales skills :,1 consumer sale skill,resp


In [98]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")
for token in doc:
    print(token.text,"-", token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])

Autonomous - amod cars NOUN []
cars - nsubj shift VERB [Autonomous]
shift - ROOT shift VERB [cars, liability]
insurance - compound liability NOUN []
liability - dobj shift VERB [insurance, toward]
toward - prep liability NOUN [manufacturers]
manufacturers - pobj toward ADP []


In [100]:
doc[0].head.pos_

'NOUN'

In [128]:
from nltk.tag import pos_tag
from nltk.tokenize import word_tokenize


In [122]:
def count_pos(txt):
    my_count = Counter()
    doc = nlp(txt)
    for token in doc:
        print(token.head.pos_)
        my_count.update((token.head.pos_))
    return my_count

In [167]:
def count_pos_2(txt):
    doc = pos_tag(word_tokenize(txt))
    #print(doc)
    my_count = Counter([j for i,j in doc])
    return my_count

In [150]:
test_str = "Autonomous cars shift insurance liability toward manufacturers"

In [151]:
count_pos_2(test_str)

Counter({'JJ': 1, 'NNS': 2, 'VB': 1, 'NN': 2, 'IN': 1})

In [165]:
test_str = str(corpus_lines_we_clean).strip('[]')
test_str = test_str.replace("'", "")

In [169]:
dict(count_pos_2(test_str))

{'FW': 84,
 'NN': 7110,
 ',': 3991,
 'CD': 1480,
 'VBP': 194,
 'JJ': 1584,
 'VBN': 125,
 'VB': 198,
 'VBG': 467,
 'VBZ': 41,
 'JJS': 33,
 'MD': 85,
 'VBD': 261,
 'IN': 109,
 'NNS': 166,
 'RB': 177,
 'JJR': 38,
 'PRP': 8,
 'WP': 9,
 'NNP': 80,
 'CC': 10,
 'RP': 5,
 'DT': 7,
 'EX': 3,
 '$': 1,
 'WRB': 1,
 'RBR': 1}

In [175]:
results = pd.DataFrame.from_dict(dict(count_pos_2(test_str)), orient='index').reset_index()

In [180]:
results.sort_values(by = 0,ascending = False)

,index,0
1,NN,7110
2,",",3991
5,JJ,1584
3,CD,1480
8,VBG,467
12,VBD,261
7,VB,198
4,VBP,194
15,RB,177
14,NNS,166


In [187]:
data['lenght'] = data.clean.apply(lambda x: len(str(x)))

In [195]:
data['noun_count'] = data.clean.apply(lambda x: count_pos_2(str(x))['NN'])
data['adjectiv_count'] = data.clean.apply(lambda x: count_pos_2(str(x))['JJ'])
data['number_count'] = data.clean.apply(lambda x: count_pos_2(str(x))['CD'])
data['verb_present_count'] = data.clean.apply(lambda x: count_pos_2(str(x))['VBG'])
data['verb_base_count'] = data.clean.apply(lambda x: count_pos_2(str(x))['VB'])
data['verb_past_count'] = data.clean.apply(lambda x: count_pos_2(str(x))['VBD'])
data['adverb_count'] = data.clean.apply(lambda x: count_pos_2(str(x))['RB'])

In [196]:
data

,original,clean,label,lenght,noun_count,adjectiv_count,number_count,verb_present_count,verb_base_count,verb_past_count,adverb_count
268,- frite arseneault,frite arseneault,company,16,1,0,0,0,0,0,1
3011,focus on retail sales of wireless,focus retail sale wireless,resp,26,3,1,0,0,0,0,0
3551,• performing database entry.,performing database entry,resp,25,2,0,0,1,0,0,0
1619,april 2018 - present,april 2018 present,date,18,2,0,1,0,0,0,0
3712,2012-2013,20122013,date,8,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
1749,"peterborough, on",peterborough,other,12,1,0,0,0,0,0,0
436,nestle india,nestle india,company,12,1,0,0,0,0,0,1
417,customer service representative,customer service representative,position,31,3,0,0,0,0,0,0
3633,1. consumer sales skills :,1 consumer sale skill,resp,21,3,0,1,0,0,0,0
